In [2]:
from __future__ import print_function

import numpy as np
import scipy.misc
import scipy.io
import tensorflow as tf
from PIL import Image

from keras.layers import Input
from keras.applications import vgg19

import keras.backend as K

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

Using TensorFlow backend.


In [3]:
class CONFIG:
    IMAGE_WIDTH = 400
    IMAGE_HEIGHT = 300
    COLOR_CHANNELS = 3
    CONTENT_WEIGHT = 5
    STYLE_WEIGHT = 100
    TOTAL_VARIATION_WEIGHT = 1.
    NOISE_RATIO = .6
    MEANS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [4]:
STYLE_LAYERS = [
    ('conv1_1', .5),
#     ('conv1_2', .2),
    ('conv2_1', .5),
#     ('conv2_2', .2),
    ('conv3_1', .5),
#     ('conv3_2', 0.2),
#     ('conv3_3', .2),
#     ('conv3_4', 0.2),
    ('conv4_1', .5),
#     ('conv4_2', 0.2),
#     ('conv4_3', 0.2),
#     ('conv4_4', .2),
    ('conv5_1', .5),
#     ('conv5_2', 0.2),
#     ('conv5_3', 0.2),
#     ('conv5_4', .2)
]

In [5]:
def reshape_and_normalize_image(image_path):
    """
    Reshape and normalize the input image (content or style)
    """
    
    image = Image.open(image_path)
    image = image.resize((CONFIG.IMAGE_WIDTH, CONFIG.IMAGE_HEIGHT))
    
    image_array = np.asarray(image, dtype='float32')
    image = np.expand_dims(image_array, axis=0)
    
    # Substract the mean to match the expected input of VGG16
    image = image - CONFIG.MEANS
    
    return image

In [6]:
def generate_noise_image(content_image, noise_ratio = CONFIG.NOISE_RATIO):
    """
    Generates a noisy image by adding random noise to the content_image
    """
    
    # Generate a random noise_image
    noise_image = np.random.uniform(-20, 20, (1, CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH,
                                              CONFIG.COLOR_CHANNELS)).astype('float32')
    
    # Set the input_image to be a weighted average of the content_image and a noise_image
    input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio)
    
    return input_image

In [7]:
content_image_path = "./images/louvre_small.jpg"
content_image = reshape_and_normalize_image(content_image_path)

style_image_path = "./images/monet.jpg"
style_image = reshape_and_normalize_image(style_image_path)

generated_image = generate_noise_image(content_image)

In [8]:
def compute_content_loss(content, generated):
    m, n_H, n_W, n_C = generated.shape

    content = np.reshape(content, [-1])
    generated = np.reshape(generated, [-1])

    return np.sum(np.square(content - generated)) / (4*n_H*n_W*n_C)

In [9]:
def gram_matrix(x):
    gram = np.dot(np.transpose(x), x)
    return gram

def style_layer_loss(style, generated):
    m, n_H, n_W, n_C = generated.shape
    
    style = np.reshape(style, (n_H*n_W, n_C))
    generated = np.reshape(generated, (n_H*n_W, n_C))
    
    S = gram_matrix(style)
    G = gram_matrix(generated)

    channels = n_C
    size = n_H*n_W

    return np.sum(np.square(S - G)) / (4. * (channels ** 2) * (size ** 2))

In [10]:
def total_loss(sess, alpha=20, beta=40):

    # Assign the input of the model to be the "content" image 
    sess.run(model["input"].assign(content_image))

    # Get content loss from output of block 4, layer 2
    out = model["conv4_2"]
    content_features = sess.run(out)
    generated_features = out
    content_loss = compute_content_loss(content_features, generated_features)
    
    # loss for style image
    style_loss = 0.
    
    # Assign the input of the model to be the "style" image 
    sess.run(model["input"].assign(style_image))
    
    for layer_name, coeff in STYLE_LAYERS:
        # Select the output tensor of the currently selected layer
        out = model[layer_name]
        style_features = sess.run(out)
        generated_features = out
        style_loss += coeff * style_layer_loss(style_features, generated_features)
    
    # Get total loss using alpha and beta
    total_loss = (content_loss*alpha) + (style_loss*beta)
    
    return content_loss, style_loss, total_loss

In [11]:
sess = tf.Session()
K.set_session(sess)

In [14]:
# Combined input for model
input_tensor = K.concatenate([content_image,
                          style_image,
                          generated_image], axis=0)

model = vgg19.VGG19(weights=None,
                    input_tensor=input_tensor,
                    include_top=False)
model.load_weights("./vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [20]:
c = sess.run(model.output[0])
s = sess.run(model.output[1])

In [24]:
c == s

array([[[ True,  True,  True, ...,  True, False,  True],
        [ True,  True,  True, ...,  True, False,  True],
        [ True,  True,  True, ...,  True, False,  True],
        ..., 
        [ True,  True,  True, ...,  True, False,  True],
        [ True,  True,  True, ...,  True, False,  True],
        [ True,  True,  True, ...,  True, False,  True]],

       [[ True,  True,  True, ...,  True, False,  True],
        [ True,  True, False, ...,  True, False,  True],
        [ True,  True,  True, ...,  True, False,  True],
        ..., 
        [ True,  True,  True, ...,  True, False,  True],
        [ True,  True,  True, ...,  True, False,  True],
        [ True,  True,  True, ...,  True, False,  True]],

       [[ True,  True,  True, ...,  True, False,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ..., 
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True, 